In [ ]:
import pandas as pd
import os
import sys

In [ ]:
homey = os.getcwd()

In [ ]:
sys.path.insert(0, 'Z:\Python projects\FishbowlAPITestProject')
import connecttest

In [ ]:
myresults = connecttest.create_connection(homey, 'poquery.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, homey, 'podata.xlsx')

In [ ]:
poData = pd.read_excel(os.path.join(homey, 'podata.xlsx'), header=0)

In [ ]:
poData['VENDOR'] = poData['VENDOR'].str.replace('COMMAESCAPE', ',')

In [ ]:
myresults = connecttest.create_connection(homey, 'soquery.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, homey, 'sodata.xlsx')

In [ ]:
soData = pd.read_excel(os.path.join(homey, 'sodata.xlsx'), header=0)

In [ ]:
soData['Customer'] = soData['Customer'].str.replace('COMMAESCAPE', ',')

In [ ]:
poExactMatch = pd.merge(poData.copy(), soData.copy(), how='left', on='SO')

In [ ]:
poNoMatch = poExactMatch[poExactMatch['Customer'].isnull()].copy()

In [ ]:
poExactMatch = poExactMatch[poExactMatch['Customer'].notnull()].copy()

In [ ]:
poNoMatch['SO'].unique().tolist()

In [ ]:
soData['compactSO'] = soData['SO'].str.slice(0,5)

In [ ]:
soData

In [ ]:
soCompact = soData.copy().drop_duplicates('compactSO')

In [ ]:
soCompact['compactSO'].iat[1]

In [ ]:
soCompact[soCompact['compactSO'] == '47401']

In [ ]:
poNoMatch.dropna(0, subset=['SO'], inplace=True)

In [ ]:
poNoMatch.reset_index(drop=True, inplace=True)

In [ ]:
len(poNoMatch[poNoMatch['SO'].str.contains('47401')])

In [ ]:
poTryMatch = pd.DataFrame()

In [ ]:
for line in range(0, len(soCompact)):
    compSO = soCompact['compactSO'].iat[line]
    actualSO = soCompact['SO'].iat[line]
    customer = soCompact['Customer'].iat[line]
    if len(poNoMatch[poNoMatch['SO'].str.contains(compSO)]) > 0:
        

In [ ]:
# screw it, I'm doing this part by hand.
writer = pd.ExcelWriter('pocheck.xlsx')
poExactMatch.to_excel(writer, 'poExact')
poNoMatch.to_excel(writer, 'poNoMatch')
writer.save()